<a href="https://colab.research.google.com/github/DoctorDroid/DS-Unit-2-Linear-Models/blob/master/Copy_of_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [20]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [14]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .5% latitudes, &
# the most extreme .5% longitudes

# df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
#         (df['price'] <= np.percentile(df['price'], 99.5)) & 
#         (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
#         (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
#         (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
#         (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

# the 'gimmie' code seemed off  so I altered it below

df = df[(df['price'] >= np.percentile(df['price'],.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], .5)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.5)) &
        (df['longitude'] >= np.percentile(df['longitude'], .5)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.5))]

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48066 entries, 0 to 49350
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bathrooms             48066 non-null  float64
 1   bedrooms              48066 non-null  int64  
 2   created               48066 non-null  object 
 3   description           46671 non-null  object 
 4   display_address       47935 non-null  object 
 5   latitude              48066 non-null  float64
 6   longitude             48066 non-null  float64
 7   price                 48066 non-null  int64  
 8   street_address        48056 non-null  object 
 9   interest_level        48066 non-null  object 
 10  elevator              48066 non-null  int64  
 11  cats_allowed          48066 non-null  int64  
 12  hardwood_floors       48066 non-null  int64  
 13  dogs_allowed          48066 non-null  int64  
 14  doorman               48066 non-null  int64  
 15  dishwasher         

In [73]:
df.sample(7)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,rooms,rm_ratio
17098,2.0,2,2016-06-24 06:01:37,BRIGHT AND SPACIOUS 2BR - STEPS TO THE HIGHLIN...,W 30th St,40.7521,-74.0011,6725,500 W 30th St,low,0,1,1,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,4.0,1.0
46839,1.0,0,2016-04-04 14:59:55,One of the best prices you will find in the Fi...,West Street,40.7094,-74.0141,2500,1 West Street,low,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1.0,0.0
40583,2.0,2,2016-05-11 02:24:27,*NO FEE* 2 Bedroom/2 Bath w/Formal Dining Room...,E 92 Street,40.7838,-73.9534,7150,115 E 92 Street,low,1,0,1,0,1,1,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,4.0,1.0
20959,1.0,0,2016-06-28 03:33:48,NO FEE --BEAUTIFUL AND LG STUDIO in Prime Mid-...,East 52th Street,40.7563,-73.9686,2275,234 East 52th Street,low,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0
46326,1.0,2,2016-04-18 01:19:03,Two bedrooms separated by a bathroom in a West...,West Houston Street,40.7288,-74.0030,2875,176 West Houston Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,2.0
16215,2.0,2,2016-06-16 02:28:35,CooperCooper.com :: Web ID #182794; Access 100...,East 94th Street,40.7826,-73.9469,4823,345 East 94th Street,low,1,1,0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,1.0
12142,1.0,1,2016-06-21 06:28:38,Dramatic 1-Bedroom in prime location with a fa...,West 43rd Street,40.7612,-73.9978,3625,560 West 43rd Street,low,1,0,1,0,1,1,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,2.0,1.0


In [23]:
# Feature engineering
df['rooms'] = df['bathrooms'] + df['bedrooms']
df['rm_ratio'] = df['bedrooms'] / df['bedrooms']

In [36]:
df.dropna(inplace=True)

In [37]:
# Train/Test split
train = df[df['created'] < '2016-06']
test = df[df['created'] >= '2016-06']

To verify


In [75]:
test

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,rooms,rm_ratio
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,2.0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,2.0
11,1.0,1,2016-06-03 03:21:22,Check out this one bedroom apartment in a grea...,W. 173rd Street,40.8448,-73.9396,1675,644 W. 173rd Street,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,1.0
14,1.0,1,2016-06-01 03:11:01,Spacious 1-Bedroom to fit King-sized bed comfo...,East 56th St..,40.7584,-73.9648,3050,315 East 56th St..,low,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,1.0
24,2.0,4,2016-06-07 04:39:56,SPRAWLING 2 BEDROOM FOUND! ENJOY THE LUXURY OF...,W 18 St.,40.7391,-73.9936,7400,30 W 18 St.,medium,1,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49305,1.0,2,2016-06-16 04:20:46,Spacious sunny 2 bedroom apartment/ Queen size...,W 175 Street,40.8456,-73.9361,2295,575 W 175 Street,low,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,2.0
49310,1.0,3,2016-06-21 06:25:35,"Soaring 40 stories into the Manhattan skyline,...",Second Avenue,40.7817,-73.9497,3995,1751 Second Avenue,high,1,0,1,0,1,1,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,4.0,3.0
49320,1.0,1,2016-06-02 13:24:18,Great deal for a one bedroom located in Prime ...,West 55th Street,40.7669,-73.9917,2727,448 West 55th Street,medium,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,1.0
49332,1.0,2,2016-06-06 01:22:44,Don't miss out on this spacious and beautiful ...,West 98th Street,40.7957,-73.9705,4850,220 West 98th Street,low,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,2.0


In [76]:
train

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,rooms,rm_ratio
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,1.0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,1.0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,4.0
5,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.0,2.0
6,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49345,1.0,1,2016-04-11 03:29:05,**89th & Park Ave! (block from the central par...,East 89th Street,40.7889,-73.9719,2179,117 East 89th Street,low,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,1.0
49346,1.0,1,2016-04-22 15:44:11,24hr Doorman Luxury building in the heart of t...,East 10th Street,40.7296,-73.9869,4500,166 2nd avenue,medium,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,1.0
49348,1.0,1,2016-04-04 18:22:34,"HIGH END condo finishes, swimming pool, and ki...",Rector Pl,40.7102,-74.0163,3950,225 Rector Place,low,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,2.0,1.0
49349,1.0,1,2016-04-16 02:13:40,Large Renovated One Bedroom Apartment with Sta...,West 45th Street,40.7601,-73.9900,2595,341 West 45th Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,1.0



# looks good, let's start engineering features.


(Moved above for efficiency. I will add the features and then split the data instead of doubling the code.)

In [ ]:
#see above

 Fit a linear regression model with at least two features.

In [77]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
features = ['rooms','bathrooms']
target = ['price']

# Creating a variable for features matrix and target vector:
X_train = train[features]
y_train = train[target]


In [78]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30244 entries, 2 to 49350
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   rooms      30244 non-null  float64
 1   bathrooms  30244 non-null  float64
dtypes: float64(2)
memory usage: 708.8 KB


In [79]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

 Get the model's coefficients and intercept.

In [80]:
model.coef_[0]

array([ 395.42970734, 1714.73109539])

In [81]:
model.intercept_[0]

435.48994854227703

Get regression metrics RMSE, MAE, and R2

In [82]:
import sklearn.metrics as metrics

In [94]:
y_pred = model.predict(X_train)
y_train = train[target]

In [95]:
mae = metrics.mean_absolute_error(y_train, y_pred)
#mse = metrics.mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse) 
      # or mse**(1\2)  
r2 = metrics.r2_score(y_train,y_pred)

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("RMSE:", rmse)
print("R2:", r2)

Results of sklearn.metrics:
MAE: 802.8348110542056
RMSE: 1216.4435699221292
R2: 0.5253817327913447


In [90]:
X_test = test[features]

In [91]:
y_pred2 = model.predict(X_test)
y_test = test[target]

In [92]:
mae = metrics.mean_absolute_error(y_test, y_pred2)
#mse = metrics.mean_squared_error(y_test, y_pred2)
rmse = np.sqrt(mse) 
      # or mse**(1\2)  
r2 = metrics.r2_score(y_test,y_pred2)

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("RMSE:", rmse)
print("R2:", r2)

Results of sklearn.metrics:
MAE: 809.9800076242461
RMSE: 1216.4435699221292
R2: 0.5374287923949217
